In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import numpy as np

# Đọc dữ liệu và tiền xử lý (từ mã của bạn)
df = pd.read_csv('cleaned_comment_data.csv')
df['text'] = df['title'].fillna('') + ' ' + df['content'].fillna('')

stopwords = set(['được', 'thì', 'có', 'mình', 'là', 'nên', 'ko', 'không', 'và', 'kì', 'cực'])

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords]
    return ' '.join(tokens)

df['text_cleaned'] = df['text'].apply(preprocess_text)

# Tạo cột sentiment
def map_sentiment(rating):
    if rating >= 4:
        return 'Tích cực'
    elif rating == 3:
        return 'Trung tính'
    else:
        return 'Tiêu cực'

df['sentiment'] = df['rating'].apply(map_sentiment)

# Chuyển đổi văn bản thành TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, max_df=0.8, min_df=5)
X = vectorizer.fit_transform(df['text_cleaned'])

# Áp dụng LDA
n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

# Gán chủ đề chính cho mỗi bình luận
topic_dist = lda.transform(X)
df['main_topic'] = topic_dist.argmax(axis=1) + 1  # Chủ đề từ 1 đến n_topics

# Hiển thị từ khóa cho mỗi chủ đề
feature_names = vectorizer.get_feature_names_out()
print("Từ khóa của các chủ đề:")
for topic_idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[-15:]]
    print(f"Chủ đề {topic_idx + 1}: {', '.join(top_words)}")

# Phân tích cảm xúc theo chủ đề
topic_sentiment = df.groupby(['main_topic', 'sentiment']).size().unstack(fill_value=0)
topic_sentiment['Tổng'] = topic_sentiment.sum(axis=1)
for sentiment in ['Tích cực', 'Trung tính', 'Tiêu cực']:
    topic_sentiment[f'{sentiment} (%)'] = (topic_sentiment[sentiment] / topic_sentiment['Tổng'] * 100).round(2)

# In kết quả dễ đọc
print("\nPhân bố cảm xúc theo chủ đề:")
print(topic_sentiment[['Tích cực', 'Trung tính', 'Tiêu cực', 'Tích cực (%)', 'Trung tính (%)', 'Tiêu cực (%)']])

# Vẽ biểu đồ
topic_sentiment[['Tích cực', 'Trung tính', 'Tiêu cực']].plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Phân bố cảm xúc theo chủ đề')
plt.xlabel('Chủ đề')
plt.ylabel('Số lượng bình luận')
plt.legend(title='Cảm xúc')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'underthesea'